# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, make_scorer
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt

# Data treatment

In [3]:
data_url = './Projeto/proj-data.csv'
data = pd.read_csv(data_url,sep=',',header=0)

hyperthyroid_conditions = ['A', 'B', 'C', 'D']
hypothyroid_conditions = ['E', 'F', 'G', 'H']
binding_protein = ['I', 'J']
general_health = ['K']
replacement_therapy = ['L', 'M', 'N']
discordant_results = ['R']

for d in range(len(data)):
    if data.at[d, "diagnoses"] in hyperthyroid_conditions :
        data.at[d, "diagnoses"] = 1
    elif data.at[d, "diagnoses"] in hypothyroid_conditions :
        data.at[d, "diagnoses"] = 2
    elif data.at[d, "diagnoses"] in binding_protein :
        data.at[d, "diagnoses"] = 3
    elif data.at[d, "diagnoses"] in general_health :
        data.at[d, "diagnoses"] = 4
    elif data.at[d, "diagnoses"] in replacement_therapy :
        data.at[d, "diagnoses"] = 5
    elif data.at[d, "diagnoses"] in discordant_results :
        data.at[d, "diagnoses"] = 6
    elif data.at[d, "diagnoses"] == "-" :
        data.at[d, "diagnoses"] = 7 # no condition
    else:
        data.at[d, "diagnoses"] = 8 # others

data.drop('T4U measured:', axis=1, inplace=True)
data.drop('TT4 measured:', axis=1, inplace=True)
data.drop('FTI measured:', axis=1, inplace=True)
data.drop('TBG measured:', axis=1, inplace=True)
data.drop('TSH measured:', axis=1, inplace=True)
data.drop('T3 measured:', axis=1, inplace=True)

data.replace('?', np.nan, inplace=True)
data.replace('f', 0, inplace=True)
data.replace('t', 1, inplace=True)

data

FileNotFoundError: [Errno 2] No such file or directory: './Projeto/proj-data.csv'

In [ ]:
#identify columns with empty values
result = {}

for j in data.values:
  count=0
  for s in j:
    if pd.isna(s):
      if data.columns[count] not in result:
        result[data.columns[count]] = 1
      else:
        result[data.columns[count]] += 1
    count+=1

for r in result.keys():
  if result[r] > 0:
    print("column " + r + " " + str(result[r]) + " missing values")

X_values = data.values[:50,:-2]

print(X_values[1][16])
print(type(X_values[1][16]))

X_values = data.iloc[:50,:-2]

print(X_values['TSH:'][1])
print(type(X_values['TSH:'][1]))


X_values.drop('TBG:',axis=1,inplace=True)
X_values.drop('T3:',axis=1,inplace=True)
X_values.drop('sex:',axis=1,inplace=True) # ASSUMINDO QUE O SEXO NÃO É RELEVANTE
X_values.drop('referral source:',axis=1,inplace=True) # ASSUMINDO QUE A FONTE DE REFERÊNCIA NÃO É RELEVANTE

print(type(X_values['TSH:'][1]))


y_value = data.iloc[:50 , -2:-1]
y_value = y_value.astype('int')

print(X_values)
print(type(X_values['TSH:'][1]))

In [ ]:
#TRAIN & TEST SET
X_train, X_test, y_train, y_test = train_test_split(X_values, y_value, test_size=0.25,random_state=0)

#SCALING
scaler = StandardScaler()
scaler.fit(X_train)
Xt_train=scaler.fit_transform(X_train)
Xt_test=scaler.fit_transform(X_test)

#IMPUTER
imputer = KNNImputer(weights="uniform")
imputer.fit(X_train)
Xt_train = imputer.transform(X_train)
Xt_test = imputer.transform(X_test)

#FEATURE SELECTION
N,M = Xt_train.shape

print(y_train)

rfr=RandomForestRegressor(random_state=0)
sel = SelectFromModel(estimator=rfr,threshold=0.015)
y_train = y_train.squeeze().ravel()
y_test = y_test.squeeze().ravel()
sel.fit(Xt_train, y_train)

print(y_train)

print("Default threshold: ", sel.threshold_)

features=sel.get_support()
Features_selected =np.arange(M)[features]

print("The features selected are columns: ", Features_selected)

nX_train=sel.transform(Xt_train)
nX_test=sel.transform(Xt_test)

score = make_scorer(matthews_corrcoef)

def  printClaStatistics(TRUTH_nfold, PREDS_nfold):   
    print("These are the training set statistics:")
    print("The Accuracy is: %7.4f" % accuracy_score(TRUTH_nfold, PREDS_nfold))
    print("The Precision is: %7.4f" % precision_score(TRUTH_nfold, PREDS_nfold, average='weighted'))
    print("The Recall is: %7.4f" % recall_score(TRUTH_nfold, PREDS_nfold, average='weighted'))
    print("The F1 score is: %7.4f" % f1_score(TRUTH_nfold, PREDS_nfold, average='weighted'))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(TRUTH_nfold, PREDS_nfold))
    print(pd.DataFrame(confusion_matrix(TRUTH_nfold, PREDS_nfold)))
    


In [ ]:
clf_rf = RandomForestClassifier(random_state=43)      
clr_rf = clf_rf.fit(Xt_train,y_train)

clf_rf_4 = RandomForestClassifier()      
clr_rf_4 = clf_rf_4.fit(Xt_train,y_train)
importances = clr_rf_4.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf_rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
 
#Print the feature ranking
print("Feature ranking:")
 
for f in range(16):
    print("%d. feature %d (%f)" % (f + 1, indices[f],  importances[indices[f]]))
 
#Plot the feature importances list:
plt.figure(1, figsize=(9,7))
plt.title("Feature importances")
plt.bar(range(Xt_train.shape[1]), importances[indices],
       color="g", yerr=std[indices], align="center")
plt.xticks(range(X_values.shape[1]), X_values.columns,rotation=90)
plt.xlim([-1, Xt_train.shape[1]])
plt.show()

# Classification Methods

## Decision Tree


In [ ]:
from sklearn import tree

print("base:")
dt = tree.DecisionTreeClassifier()
dt.fit(nX_train , y_train)
preds = dt.predict(nX_test)
printClaStatistics(y_test, preds)

print()
print("best:")
cs = ["gini", "entropy"]
md = [None,*range(3, 30)]
msl=[*range(1,15)]
mss=[*range(2,15)]
param_grid = {"criterion": cs, "max_depth": md, "min_samples_leaf": msl, "min_samples_split": mss}
gs = GridSearchCV(estimator=dt, param_grid=param_grid, scoring=score)
gs.fit(nX_train , y_train)
preds=gs.best_estimator_.predict(nX_test)
printClaStatistics(y_test, preds)

print("best criterion : " ,gs.best_estimator_.criterion)
print("best max_depth : " ,gs.best_estimator_.max_depth)
print("best min_samples_leaf : " ,gs.best_estimator_.min_samples_leaf)
print("best min_samples_split : " ,gs.best_estimator_.min_samples_split)


## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
print("base:")
mdl=GaussianNB()
mdl.fit(nX_train , y_train)
preds = mdl.predict(nX_test)

printClaStatistics(y_test, preds)

## K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def gaussian(dsts):
    kernel_width = .5
    weights = np.exp(-(dsts**2)/kernel_width)
    return weights

print("base:")
knn = KNeighborsClassifier()
knn.fit(nX_train , y_train)
preds = knn.predict(nX_test)
printClaStatistics(y_test, preds)

print()
print("best:")
nn=[3,5,7,9,11]
w=["distance","uniform",gaussian]

param_grid = {"n_neighbors":nn,"weights":w}
gs = GridSearchCV(estimator=knn, param_grid=param_grid, scoring=score)
gs.fit(nX_train , y_train)
preds=gs.best_estimator_.predict(nX_test)

printClaStatistics(y_test, preds)

print("best n_neighbors : " ,gs.best_estimator_.n_neighbors)
print("best weights : " ,gs.best_estimator_.weights)


## SVM

In [ ]:
from sklearn.svm import SVC

# Compute the minimum value per feature on training set
min_on_training = nX_train.min(axis=0)
# Compute the range of each feature (max - min) on  training set
range_on_training = (nX_train - min_on_training).max(axis=0)

# subtract the min, divide by range
# afterward, min=0 and max=1 for each feature
X_train_scaled = (nX_train - min_on_training) / range_on_training

# use THE SAME transformation on the test set,
# using min and range of the training set. See Chapter 3 (unsupervised learning) for details.
X_test_scaled = (nX_test - min_on_training) / range_on_training

clf = SVC(C = 1, gamma=0.1)
svc= clf.fit(X_train_scaled, y_train)
preds = svc.predict(X_test_scaled)

printClaStatistics(y_test, preds)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=9,criterion='entropy')
rf.fit(nX_train, y_train)
preds=rf.predict(nX_test)
printClaStatistics(y_test, preds)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=50,random_state=0)
abc.fit(nX_train, y_train)
preds=abc.predict(nX_test)

printClaStatistics(y_test, preds)

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

print("base:")
gbc = GradientBoostingClassifier()
gbc.fit(nX_train, y_train)
preds = gbc.predict(nX_test)
printClaStatistics(y_test, preds)

print()
print("best:")

ne = [*range(10,200,10)]
lr = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
param_grid = {"n_estimators":ne,"learning_rate":lr}
gs = GridSearchCV(estimator=gbc, param_grid=param_grid, scoring=score)
gs.fit(nX_train , y_train)
preds=gs.best_estimator_.predict(nX_test)

printClaStatistics(y_test, preds)

print("best n_estimators : " ,gs.best_estimator_.n_estimators)
print("best learning_rate : " ,gs.best_estimator_.learning_rate)

## XGBoost

In [ ]:
from xgboost import XGBClassifier
print("base:")
bst = XGBClassifier()
bst.fit(nX_train, y_train)
preds = bst.predict(nX_test)
printClaStatistics(y_test, preds)